### Steps for CNN of ASL Dataset

Load ASL training and test datasets 
- don't know how to process the image from individual pixels to torch or matplotlib
Define a Convolutional Neural Network
- working on understand different parts of torch architecture steps
Define a loss function
Train the network on the training data
Test the network on the test data

Visualize # of each letter in datasets(Bar graph to show samples in each class)
Randomly generate 1 sample from each class
Show correctly classified and wrongly classified to show why some are easier to classify vs harder to classify

#### Dataset Origin 
https://www.kaggle.com/datamunge/sign-language-mnist

#### Pytorch tutorial/inspiration
https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

In [1]:
%matplotlib inline
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns
from torch.utils.data import DataLoader, Dataset
import torch.utils.data as data_utils
import torchvision
import torchvision.transforms as T
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [2]:
#read in data
df_test = pd.read_csv('./sign_mnist_test.csv')
df_train = pd.read_csv('./sign_mnist_train.csv')

In [3]:
df_train.head()
df_test.head()
print(df_train.shape)
print(df_test.shape)

(27455, 785)
(7172, 785)


In [4]:
df_train.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,3,107,118,127,134,139,143,146,150,153,...,207,207,207,207,206,206,206,204,203,202
1,6,155,157,156,156,156,157,156,158,158,...,69,149,128,87,94,163,175,103,135,149
2,2,187,188,188,187,187,186,187,188,187,...,202,201,200,199,198,199,198,195,194,195
3,2,211,211,212,212,211,210,211,210,210,...,235,234,233,231,230,226,225,222,229,163
4,13,164,167,170,172,176,179,180,184,185,...,92,105,105,108,133,163,157,163,164,179


In [6]:
#separate labels (letter classes) from training data set
#drop label from rest of pixels
labels_train = df_train['label']
df_train = df_train.drop('label',axis = 1)

KeyError: 'label'

In [7]:
# #separate labels (letter classes) from test data set
#drop label from rest of pixels
labels_test = df_test['label']
df_test = df_test.drop('label',axis = 1)

### Data visualizations 
- visualize # of each letter in total dataset
- (randomly) generate sample
- more?

In [ ]:
# create dataframe with all labels to make a bargraph
frames = [labels_train,labels_test]
df_all_labels = pd.concat(frames)
print(df_all_labels.shape)
df_all_labels.head()

In [ ]:
# create a bar graph with number of each labels
'''Want to change so that 0,1,2,3 etc reads A,B,C,D etc'''

ax = sns.barplot(x=df_all_labels.value_counts().index, y=df_all_labels.value_counts())
ax.set(ylabel = '# counts', xlabel = 'Alphabet Letters' )

In [ ]:
df_all_labels.value_counts()

In [ ]:
#convert each image to a 28x28 array
testimg = np.array(df_train[4:5]).reshape(28,28)
plt.imshow(testimg,cmap = 'gray')

### Converting data to tensor

In [ ]:
# convert pandas df to numpy array and reshape into 28x28 array as prep for picture
np_train = df_train.to_numpy() #training dataset
np_train = np_train.reshape(np_train.shape[0],28,28)
np_test = df_test.to_numpy() # testing dataset
np_test = np_test.reshape(np_test.shape[0],28,28)

In [ ]:
#show random image
random_image = random.randint(0,len(np_train))
plt.imshow(np_train[random_image], cmap = 'gray')
plt.title(f"Training Example #{random_image}")
plt.xlabel(labels_train[random_image]) #xlabel = letter of alphabet

In [ ]:
# train_loader = data_utils.DataLoader(train, batch_size = 10, shuffle = True)

In [29]:
#converting to tensor with dataloader
batch_size = 10

train = data_utils.TensorDataset(torch.Tensor(np.array(df_train).reshape(-1,28,28)), torch.Tensor(np.array(labels_train)))
test = data_utils.TensorDataset(torch.Tensor(np.array(df_test).reshape(-1,28,28)), torch.Tensor(np.array(labels_test)))

trainloader = DataLoader(train, batch_size = batch_size,
                         shuffle = True, num_workers = 2)
testloader = DataLoader(test, batch_size = batch_size,
                       shuffle = False, num_workers = 2)

In [ ]:
#COME BACK LATER
def imshow(img):
#     img = img / 2 + 0.5     # unnormalize
#     npimg = img.numpy()
#     plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % labels_train[j] for j in range(batch_size)))

In [39]:
#define neural network
class Net(nn.Module):
    
    def __init__(self):
        super().__init__()
        # 1 input channel (gray), 6 output channels, 3x3 square convolution
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.pool = nn.MaxPool1d(2)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(16 * 3 * 3, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 24)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [11]:
#define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [40]:
# train network
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        print(data)
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[tensor([[[164., 164., 166.,  ..., 167., 166., 165.],
         [167., 167., 168.,  ..., 168., 168., 167.],
         [169., 170., 171.,  ..., 170., 169., 168.],
         ...,
         [214., 215., 216.,  ...,  87.,  96., 204.],
         [215., 216., 218.,  ...,  81., 130., 221.],
         [217., 218., 218.,  ...,  77., 181., 222.]],

        [[ 15.,  32.,  44.,  ..., 137., 137., 138.],
         [ 17.,  34.,  46.,  ..., 137., 138., 139.],
         [ 20.,  35.,  49.,  ..., 138., 140., 140.],
         ...,
         [ 52.,  64.,  85.,  ..., 164., 165., 166.],
         [ 52.,  65.,  85.,  ..., 164., 166., 166.],
         [ 52.,  67.,  85.,  ..., 165., 166., 166.]],

        [[133., 136., 138.,  ..., 168., 168., 168.],
         [134., 136., 140.,  ..., 168., 168., 168.],
         [134., 137., 142.,  ..., 170., 170., 170.],
         ...,
         [147., 150., 153.,  ..., 187., 187., 187.],
         [147., 150., 153.,  ..., 187., 187., 187.],
         [147., 151., 153.,  ..., 188., 188., 188.]]

RuntimeError: Expected 4-dimensional input for 4-dimensional weight [6, 1, 3, 3], but got 3-dimensional input of size [10, 28, 28] instead

In [16]:
len(labels_train.unique())

24

In [ ]:
# class ASLDataset(Dataset):
    
#     def __init__(self,X,y):
#         self.X = X # X = training images
#         self.y = y # y = labels
        
#     def __len__(self):
#         return len(self.X)
    
#     def __getitem__(self,index):
#         image = self.X[index]
#         X = self.transform(image)
#         return X
    

In [ ]:
# image_size = 28
# transform = T.Compose([T.ToPILImage(),
#                       T.Resize(image_size),
#                       T.ToTensor()])

In [ ]:
# batch_size = 20
# transformed_dataset = ASLDataset(np_train,labels_train)
# train_dl = DataLoader(transformed_dataset,batch_size, shuffle = True,
#                      num_workers = 3)


In [ ]:
# def show_images(images, nmax=64):
#     fig, ax = plt.subplots(figsize=(8, 8))
#     ax.set_xticks([]); ax.set_yticks([])
#     ax.imshow(make_grid((images.detach()[:nmax]), nrow=8).permute(1, 2, 0))
# def show_batch(dl, nmax=64):
#     for images in dl:
#         show_images(images, nmax)
#         break

In [ ]:
# show_batch(train_dl)